In [1]:
from ipyparallel import Client
import os

rc = Client()
data_path = os.path.abspath("data")
res_path = os.path.abspath("results")

rc[:].push(dict(
    data_path=data_path,
    res_path=res_path,
))

dview = rc.load_balanced_view()

#dview = rc.direct_view('all')
print 'n of workers:', len(dview)
print len(rc)

n of workers: 10
10


In [2]:
print dview.queue_status()

{0: {'queue': 0, 'completed': 1, 'tasks': 0}, 1: {'queue': 0, 'completed': 1, 'tasks': 0}, 2: {'queue': 0, 'completed': 1, 'tasks': 0}, 3: {'queue': 0, 'completed': 1, 'tasks': 0}, 4: {'queue': 0, 'completed': 1, 'tasks': 0}, 5: {'queue': 0, 'completed': 1, 'tasks': 0}, 6: {'queue': 0, 'completed': 1, 'tasks': 0}, 7: {'queue': 0, 'completed': 1, 'tasks': 0}, 8: {'queue': 0, 'completed': 1, 'tasks': 0}, 9: {'queue': 0, 'completed': 1, 'tasks': 0}, 'unassigned': 0}


In [3]:
%%px --local
import pandas as pd
from string import strip
from goatools.base import download_go_basic_obo
from goatools.associations import read_associations
from goatools.go_enrichment import GOEnrichmentStudy
from goatools.obo_parser import GODag
import scipy.cluster.hierarchy as sch
import numpy as np
from string import strip

in_go=data_path+'/go-basic.obo'
in_assoc=data_path+'/associations.txt'
gene_found = [strip(n.split('\t')[0]) for n in open(data_path+'/in_df.txt').read().split('\n')]
obodag = GODag(in_go)
geneid2gos = read_associations(in_assoc)
goeaobj = GOEnrichmentStudy(
            gene_found, # List of mouse protein-coding genes
            geneid2gos, # geneid/GO associations
            obodag, # Ontologies
            propagate_counts = False,
            alpha = 0.05, # default significance cut-off
            methods = ['fdr_bh']) 


load obo file D:\git\go_term_clustering\data/go-basic.obo
D:\git\go_term_clustering\data/go-basic.obo: fmt(1.2) rel(2017-06-09) 48,816 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 2,595 out of  3,119 population items found in association
[stdout:0] 
load obo file D:\git\go_term_clustering\data/go-basic.obo
D:\git\go_term_clustering\data/go-basic.obo: fmt(1.2) rel(2017-06-09) 48,816 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 2,595 out of  3,119 population items found in association
[stdout:1] 
load obo file D:\git\go_term_clustering\data/go-basic.obo
D:\git\go_term_clustering\data/go-basic.obo: fmt(1.2) rel(2017-06-09) 48,816 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 2,595 out of  3,119 population items found in association
[stdout:2] 
load obo file D:\git\go_term_clustering\data/go-basic.obo
D:\git\go_term_clustering\data/go-basic.obo: fmt(1.2) rel(2017-06-09) 48,816 GO Term

In [4]:
def score(in_genes, index_res, cut_distance, cluster_id):
    def count(goea_results_sig):
            res = {'BP' : 0,
                   'CC' : 0,
                   'MF' : 0}
            for item in goea_results_sig:
                item_list = str(item).split()            
                go_type = item_list[1] 
                res[go_type]+=1
            return res   
    goea_results_all = goeaobj.run_study(in_genes)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    res = count(goea_results_sig)
    if sum(res.values())>0:
        res_file = open(res_path+'/'+'res_'+str(cut_distance)+'_'+str(cluster_id)+'.txt','w')
        res_file.write('\t'.join(['cut_distance','cluster_id','CC','MF','BP'])+'\n')
        res_file.write('\t'.join( [str(n) for n in [cut_distance,cluster_id,res['CC'],res['MF'],res['BP']]] ))
        res_file.close()
    #return count(goea_results_sig)

In [5]:
def test():
    geneids_study = [strip(n) for n in open('test/protein_complexes/T_complex.txt').read().split('\n')]
    score(geneids_study, 'test_local', 0.04, 2)
    rc[1].apply(score, geneids_study, 'test_cluster', 0.05, 3)
#test()

In [7]:
clustering_df = pd.DataFrame.from_csv('res_clustering.csv')
print clustering_df.shape
#print clustering_df.head()

(3117, 419)


In [ ]:
import time
import sys
start_time = time.time()
#res = []

for index, col in enumerate(clustering_df.columns):
    in_data = clustering_df[col].value_counts()
    in_data = in_data[in_data>=2]
    cut_distance = col.split('_')[-1]
    print index, col, cut_distance
    for cluster in in_data.index.values:
        genes = clustering_df[col][clustering_df[col]==cluster]
        genes = genes.index.values
        #print cluster, len(genes)
        #print 'send', index, cut_distance, cluster, 'to controller'
        dview.apply(score, genes, index, cut_distance, cluster)
        test = 0
        a=0
        while 1:
            #send more job only if the queue is less than 100
            #help to do not run out of memory
            if dview.queue_status()['unassigned']<100:            
                break
            else:
                a+=1
                #print a, 'unassigned:', dview.queue_status()['unassigned']
                #print 'waited for ', a*1, 'sec'
                sys.stdout.flush()
                time.sleep(1)            
                pass        


        
print 'end after : ', time.time() - start_time

0 clusters_0.0 0.0
1 clusters_0.06 0.06
2 clusters_0.09 0.09
3 clusters_0.1 0.1
4 clusters_0.11 0.11
5 clusters_0.12 0.12
6 clusters_0.13 0.13
7 clusters_0.14 0.14
8 clusters_0.15 0.15
9 clusters_0.16 0.16
10 clusters_0.17 0.17
11 clusters_0.18 0.18
12 clusters_0.19 0.19
13 clusters_0.2 0.2
14 clusters_0.21 0.21
15 clusters_0.22 0.22
16 clusters_0.23 0.23
17 clusters_0.24 0.24
18 clusters_0.25 0.25
19 clusters_0.26 0.26
20 clusters_0.27 0.27
